In [ ]:
__author__ = "Kristian Olsen / NMBU"
__email__ = "kristian.olsen@nmbu.no"

# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, make_scorer
from sklearn.metrics import accuracy_score, classification_report



# Load data

In [ ]:

# Load data from csv, the xlsx is beforehand converted into .csv for easier management
df = pd.read_csv('/content/Cube-data_Tberg_total_24032021.csv', sep = ';')

# show first rows and information to get initial understanding of the data
print(df.head(), df.info())
print("Shape:", df.shape)

# Data processing and feature engineering

In [ ]:
# rename features from Norwegian to English
rename_dict= {'fra-sone':'origin-zone',
              'til-sone':'destination-zone',
              'fra-Grk':'origin-area',
              'Til-Grk':'destination-area',
              'Reiser':'Trips',
              'TotBef':'TotPop',
              'Arb':'Work',
              'Handel':'Retail',
              'FriHenPriv':'RecDelPriv',
              'Gange':'Walk',
              'Sykkel':'Bicycle',
              'Bil':'Car',
              'Bilpass':'Carpass'
              }

df = df.rename(columns = rename_dict)

# drop columns that origin from travel survey and being duplicate features
df = df.drop(['origin-area', # duplicate information
              'destination-area', # duplicate information
              'Ndays', # irrelevant
              'Walk', # origins from travel survey
              'Bicycle', # origins from travel survey
              'Car', # origins from travel survey
              'Carpass', # origins from travel survey
              'PT'], # origins from travel survey
              axis = 1
             )

# convert features from data object to numerical
object_columns = ['Trips', 'Distance', 'TotPop']

for column in object_columns:
  df[column] = df[column].str.replace(',', '.').astype(float)
  df[column] = pd.to_numeric(df[column])

print(df.head()) # control the renaming

# Check for missing values and duplicates
print(df.isnull().sum())
print(np.isnan(df).sum())
print("Has duplicates:", df.duplicated().any())

In [ ]:
# Correlation matrix for feature engineering
df_corr = df.drop('destination-zone', axis=1)
corr_matrix = df_corr.corr()

mask = np.zeros_like(corr_matrix, dtype=bool)
mask[np.triu_indices_from(mask, k=1)] = True

# Create a Correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title('Correlation Matrix')
plt.show()

## Statistical analysis

In [ ]:
# simple statistical overview of all features
df.describe()

# Further statistical analysis was conducted directly in the excel file

## Principal Component Analysis

In [ ]:
# Initializing Loading-plot for whole dataset
df_pca = df.drop('destination-zone', axis=1)
scaler = StandardScaler()
data_standardized = scaler.fit_transform(df_pca.T)

pca = PCA(n_components=2)  # Reduce to 2 dimensions
principal_components = pca.fit_transform(data_standardized)

feature_names = ['Origin-zone', 'Trips', 'Distance','TotPop','Work','Retail','RecDelPriv']

# Plotting the PCA
plt.figure(figsize=(8, 6))
for i, (x, y) in enumerate(principal_components):
    plt.scatter(x, y, marker='o', label=f'{feature_names[i]}')  # Plot points
    plt.text(x, y, feature_names[i], fontsize=12)

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Plot')
plt.grid(True)
plt.show()
print(f"Explained variance by component: {pca.explained_variance_ratio_}")


In [ ]:
# Initialize score plot of n = 10 000 samples from the dataset
df_pca_2 = df
df_pca_sample = df_pca_2.sample(n=10000, random_state=42)
unique_labels_3 = df_pca_sample['destination-zone'].tolist()
df_pca_sample = df_pca_sample.drop('destination-zone', axis=1)

scaler = StandardScaler()
data_standardized = scaler.fit_transform(df_pca_sample)

pca = PCA(n_components=2)  # Reduce to 2 dimensions
principal_components = pca.fit_transform(data_standardized)

# Plotting the PCA
plt.figure(figsize=(8, 6))
for i, (x, y) in enumerate(principal_components):
    plt.scatter(x, y, marker=None,label=f'{unique_labels_3[i]}')
    plt.text(x, y, unique_labels_3[i], fontsize=10)

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Plot')
plt.grid(True)
plt.show()
print(f"Explained variance by component: {pca.explained_variance_ratio_}")

In [ ]:
# Initialize loading plot of n = 10 000 samples from the dataset
df_pca_3 = df
df_pca_sample_2 = df_pca_3.sample(n=10000, random_state=42)
data_standardized = scaler.fit_transform(df_pca_sample.T)

pca = PCA(n_components=2)  # Reduce to 2 dimensions
principal_components = pca.fit_transform(data_standardized)

feature_names = ['Origin-zone', 'Trips', 'Distance','TotPop','Work','Retail','RecDelPriv']

# Plotting the PCA
plt.figure(figsize=(8, 6))
for i, (x, y) in enumerate(principal_components):
    plt.scatter(x, y, marker='o', label=f'{feature_names[i]}')  # Plot points
    plt.text(x * 1.05, y * 1.05, feature_names[i], fontsize=12)

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Plot')
plt.grid(True)
plt.show()
print(f"Explained variance by component: {pca.explained_variance_ratio_}")

In [ ]:
# Intialize score plot for unique destination zones
unique_df = df.drop_duplicates(subset='destination-zone', keep='first')
df_pca_unique = unique_df.drop('destination-zone', axis=1)
scaler = StandardScaler()
data_standardized = scaler.fit_transform(df_pca_unique)

pca = PCA(n_components=2)  # Reduce to 2 dimensions
principal_components = pca.fit_transform(data_standardized)

unique_labels = df['destination-zone'].unique().tolist()

# Plotting the PCA
plt.figure(figsize=(8, 6))
for i, (x, y) in enumerate(principal_components):
    plt.scatter(x, y, marker=None,label=f'{unique_labels[i]}')  # Plot points
    plt.text(x, y, unique_labels[i], fontsize=10)

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Plot')
plt.grid(True)
plt.show()
print(f"Explained variance by component: {pca.explained_variance_ratio_}")

In [ ]:
# Intialize loading plot for unique destination zones
unique_df = df.drop_duplicates(subset='destination-zone', keep='first')
df_pca_unique = unique_df.drop('destination-zone', axis=1)
scaler = StandardScaler()
data_standardized = scaler.fit_transform(df_pca_unique.T)

pca = PCA(n_components=2)  # Reduce to 2 dimensions
principal_components = pca.fit_transform(data_standardized)

feature_names = ['Origin-zone', 'Trips', 'Distance','TotPop','Work','Retail','RecDelPriv']

# Plotting the PCA
plt.figure(figsize=(8, 6))
for i, (x, y) in enumerate(principal_components):
    plt.scatter(x, y, marker='o', label=f'{feature_names[i]}')  # Plot points
    plt.text(x * 1.05, y * 1.05, feature_names[i], fontsize=12)

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Plot')
plt.grid(True)
plt.show()
print(f"Explained variance by component: {pca.explained_variance_ratio_}")

# Remove outliers and violin plots

In [ ]:
# Violin plots of data
data_to_plot = df.iloc[:, 2:8]

# Set up a matplotlib figure and array of axes, with 1 row and enough columns
fig, axes = plt.subplots(nrows=1, ncols=len(data_to_plot.columns), figsize=(15, 5))  # Adjust figsize as needed

# Loop through each subplot axis and column data
for ax, (colname, coldata) in zip(axes, data_to_plot.items()):
    sns.violinplot(y=coldata, ax=ax)  # Use 'y' to specify the data
    ax.set_title(colname)  # Set title to column name

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
def remove_outliers(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR

    # Filter out outliers
    df_filtered = df[(df[feature] <= upper_bound)]
    return df_filtered

feature_list = ['Trips','Distance','Work','Retail','RecDelPriv']

# Applying the function to each of the features
features = feature_list
for feature in features:
    df = remove_outliers(df, feature)

# Second visualization of violin plots to further investigate skewedness
data_to_plot = df.iloc[:, 2:8]

# Set up a figure
fig, axes = plt.subplots(nrows=1, ncols=len(data_to_plot.columns), figsize=(15, 5))

# Go through each subplot and column data
for ax, (colname, coldata) in zip(axes, data_to_plot.items()):
    sns.violinplot(y=coldata, ax=ax)  # Use 'y' to specify the data
    ax.set_title(colname)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
def remove_outliers(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 0.5 * IQR

    # Filter out outliers
    df_filtered = df[(df[feature] <= upper_bound)]
    return df_filtered

feature_list = ['Trips','Work','Retail','RecDelPriv']

# Applying the function
features = feature_list
for feature in features:
    df = remove_outliers(df, feature)

# A third visualization with violin plots
data_to_plot = df.iloc[:, 2:8]

# Set up a figure
fig, axes = plt.subplots(nrows=1, ncols=len(data_to_plot.columns), figsize=(15, 5))  # Adjust figsize as needed

# Go through each subplot axis and column data
for ax, (colname, coldata) in zip(axes, data_to_plot.items()):
    sns.violinplot(y=coldata, ax=ax)
    ax.set_title(colname)

# Show the plot
plt.tight_layout()
plt.show()

# Data modelling and hyperparameter tuning

In [ ]:
# Pipeline for Logistic Regression
pipe_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])
param_grid_lr = {
    'classifier__C': [0.01, 0.1, 1, 10, 100],
    'classifier__solver': ['liblinear', 'lbfgs']
}

# Pipeline for SVC
pipe_svc = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])
param_grid_svc = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf']
}

# Pipeline for Random Forest
pipe_rf = Pipeline([
    ('classifier', RandomForestClassifier())
])
param_grid_rf = {
    'classifier__n_estimators': [10, 50, 100, 200],
    'classifier__max_depth': [None, 10, 20, 30]
}

# Pipeline for Gaussian Naive Bayes
pipe_nb = Pipeline([
    ('classifier', GaussianNB())
])

param_grid_nb = {}

# seperate training data from response variable
X = df.drop('destination-zone', axis=1)
y = df['destination-zone']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def perform_grid_search(pipe, param_grid, X_train, y_train, X_test, y_test):
    grid_search = GridSearchCV(pipe, param_grid, cv=2, verbose=1, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    print("Best parameters:", grid_search.best_params_)
    print("Best cross-validation score: {:.3f}".format(grid_search.best_score_)) #shows accuracy performance

    # Evaluate on the test set
    y_pred = grid_search.predict(X_test)

    # Calculate Metrics
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    mcc = matthews_corrcoef(y_test, y_pred)

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-Score: {f1:.2f}")
    print(f"MCC: {mcc}")
    print(classification_report(y_test, y_pred))

# Apply the function to each classifier
print("Logistic Regression Results:")
perform_grid_search(pipe_lr, param_grid_lr, X_train, y_train, X_test, y_test)

print("SVC Results:")
perform_grid_search(pipe_svc, param_grid_svc, X_train, y_train, X_test, y_test)

print("Random Forest Results:")
perform_grid_search(pipe_rf, param_grid_rf, X_train, y_train, X_test, y_test)

print('Gaussian Naive Bayes Results:')
perform_grid_search(pipe_nb, param_grid_nb, X_train, y_train, X_test, y_test)


# Evaluation: Five-fold cross-validation

In [ ]:
# Dictionaries with best hyperparameters
lr_best_params = {
    'classifier__C': [100],
    'classifier__solver': ['lbfgs']
}
svc_best_params = {
    'classifier__C': [10],
    'classifier__kernel': ['linear']
}
rf_best_params = {
    'classifier__n_estimators': [200],
    'classifier__max_depth': [30]
}

# Model Selection with best parameters
models = {
    'LogisticRegression': LogisticRegression(C=100, max_iter=1000, solver='lbfgs' ),
    'SVC': SVC(C=10, kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=200, max_depth=30),
    'Naive Bayes': GaussianNB()
}

# Training and Validation
results = {}
scoring = {
    'accuracy': 'accuracy',
    'mcc': make_scorer(matthews_corrcoef),
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1_score': make_scorer(f1_score, average='macro'),
}
for name, model in models.items():
    # Create a pipeline with preprocessing and the model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Feature scaling
        ('model', model)
    ])

    # Cross-validation
    cv_results = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring, return_train_score=False)

    # Training
    pipeline.fit(X_train, y_train)

    # Testing
    y_pred = pipeline.predict(X_test)

    # Evaluation metrics on the test data
    test_scores = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='macro'),
        'Recall': recall_score(y_test, y_pred, average='macro'),
        'F1-score': f1_score(y_test, y_pred, average='macro'),
        'MCC': matthews_corrcoef(y_test, y_pred),

    }

    # Combine Cross validation  mean and standard deviation in results
    results[name] = {
        'CV Accuracy (mean)': np.mean(cv_results['test_accuracy']),
        'CV Accuracy (std)': np.std(cv_results['test_accuracy']),
        'CV Precision (mean)': np.mean(cv_results['test_precision']),
        'CV Precision (std)': np.std(cv_results['test_precision']),
        'CV Recall (mean)': np.mean(cv_results['test_recall']),
        'CV Recall (std)': np.std(cv_results['test_recall']),
        'CV F1-score (mean)': np.mean(cv_results['test_f1_score']),
        'CV F1-score (std)': np.std(cv_results['test_f1_score']),
        'CV MCC (mean)': np.mean(cv_results['test_mcc']),
        'CV MCC (std)': np.std(cv_results['test_mcc']),
        **test_scores
    }

    # Analyze feature importance
    if name == 'LogisticRegression':
      feature_importance_lr = pd.Series(model.coef_[0], index=X_train.columns)
      feature_importance_lr_sorted = feature_importance_lr.abs().sort_values(ascending=False)

      print("Logistic Regression Feature Importance:")
      print(feature_importance_lr_sorted)

    elif name == 'SVC':
        feature_importance_svc = pd.Series(model.coef_[0], index=X_train.columns)
        feature_importance_svc_sorted = feature_importance_svc.abs().sort_values(ascending=False)

        print("SVC Feature Importance:")
        print(feature_importance_svc_sorted)
    elif name == 'RandomForest':
        # Get feature importances
        importances = model.feature_importances_

        # Sort feature importances in descending order
        indices = np.argsort(importances)[::-1]

        # Print feature ranking
        print("Feature ranking:")
        print(model)
        print("Feature ranking:")
        for f in range(X_train.shape[1]):
          print(f"{f + 1}. Feature {indices[f]} ({X_train.columns[indices[f]]}): {importances[indices[f]]}")

    elif name == 'Naive Bayes':
        feature_variance = X_train.groupby(y).var()

        print("GaussianNB Feature Variance:")
        print(feature_variance)
    else:
      pass
